In [7]:
import torch
import ray
import gym
from IPython import display
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
breakout_env = gym.make('Breakout-v0')

ob_space = breakout_env.observation_space
ac_space = breakout_env.action_space

initial_state = breakout_env.reset()

In [4]:
from ray.rllib.policy.policy import Policy

class RandomPolicy(Policy):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        # TODO: Set some instance variables here
        # Note: the super constructor will set the instance variables `action_space` and `observation_space` for you
        pass
    
    def compute_actions(self,
                        obs_batch,
                        state_batches=None,
                        prev_action_batch=None,
                        prev_reward_batch=None,
                        **kwargs):
        # Randomly sample an action
        pass
    
    def learn_on_batch(self, samples):
        """No learning."""
        return {}
    
    def compute_log_likelihoods(self,
                                actions,
                                obs_batch,
                                state_batches=None,
                                prev_action_batch=None,
                                prev_reward_batch=None):
        # Return logits from a uniform distribution
        pass
    def get_weights(self):
        """
        No-op to keep rllib from breaking, won't be necessary in future rllib releases
        """
        pass

    def set_weights(self, weights):
        """
        No-op to keep rllib from breaking
        """
        pass

In [5]:
from ray.rllib.env.multi_agent_env import MultiAgentEnv
class BreakoutRllib(MultiAgentEnv):
    
    def __init__(self, *args, **kwargs):
        self._env = gym.make('Breakout-v0')
        self.action_space = self._env.action_space
        self.observation_space = self._env.observation_space
        
        
    def step(self, action):
        obs, reward, done, info = self._env.step(action)
        return { "random" : obs }, reward, done, info
    
    def reset(self):
        return self._env.reset()

In [6]:
# Some ray setup
from ray.rllib.agents.ppo.ppo import PPOTrainer
from ray.tune.registry import register_env
ray.init()

# Register the gym environment (Note that this is different than the gym registry!)
register_env("breakout",
             lambda _: BreakoutRllib())
obs_space = breakout_env.observation_space
act_space = breakout_env.action_space

config = config={
        "multiagent": {
            "policies": {
                "random": (RandomPolicy, obs_space, act_space, {}),
            },
            "policy_mapping_fn": (
                lambda agent_id: "random"),
        },
        "num_workers" : 1,
        "framework": "torch"
}

trainer = PPOTrainer(env="breakout", config=config)

2020-10-04 22:14:15,766	INFO resource_spec.py:231 -- Starting Ray with 0.88 GiB memory available for workers and up to 0.45 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-04 22:14:16,845	INFO services.py:1193 -- View the Ray dashboard at localhost:8265
2020-10-04 22:14:16,850	WARNING services.py:1567 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This may slow down performance! You may be able to free up space by deleting files in /dev/shm or terminating any running plasma_store_server processes. If you are inside a Docker container, you may need to pass an argument with the flag '--shm-size' to 'docker run'.
2020-10-04 22:14:16,884	WARNING services.py:1567 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This may slow down performance! You may be able to free up space by deleting files 

NameError: name 'breakout_env' is not defined